In [2]:
pip install geojson

Note: you may need to restart the kernel to use updated packages.


In [3]:
%conda install folium

Retrieving notdone
Channels:
 - defaults
Platform: osx-64
doneecting package metadata (repodata.json): - 
doneing environment: | 

# All requested packages already installed.


Note: you may need to restart the kernel to use updated packages.


In [17]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
import folium
from folium.plugins import FastMarkerCluster

In [20]:
nash_list = pd.read_csv('../data/listings.csv')

In [21]:
nash_list.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9585 entries, 0 to 9584
Data columns (total 79 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   id                                            9585 non-null   int64  
 1   listing_url                                   9585 non-null   object 
 2   scrape_id                                     9585 non-null   int64  
 3   last_scraped                                  9585 non-null   object 
 4   source                                        9585 non-null   object 
 5   name                                          9585 non-null   object 
 6   description                                   9394 non-null   object 
 7   neighborhood_overview                         6214 non-null   object 
 8   picture_url                                   9585 non-null   object 
 9   host_id                                       9585 non-null   i

In [140]:
nash_list[['latitude','longitude']].head(50)

,latitude,longitude
0,36.171430,-86.735700
1,36.124660,-86.812690
2,36.189400,-86.591620
3,36.131220,-86.800660
4,36.160760,-86.591510
5,36.162960,-86.591130
6,36.162360,-86.590050
7,36.175800,-86.799500
8,36.161220,-86.590070
9,36.164990,-86.750540


In [103]:
nash_list.loc[nash_list['minimum_minimum_nights']>=30]

,id,listing_url,scrape_id,last_scraped,source,name,description,neighborhood_overview,picture_url,host_id,...,review_scores_communication,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,6422,https://www.airbnb.com/rooms/6422,20250315060229,2025-03-15,city scrape,Nashville Charm,30 day plus rental - book for one month and th...,Historic East Nashville is home to many new an...,https://a0.muscache.com/pictures/miso/Hosting-...,12172,...,4.96,4.92,4.98,NaN,f,1,0,1,0,3.46
2,59576,https://www.airbnb.com/rooms/59576,20250315060229,2025-03-15,city scrape,Large Main Suite near Lake *ladies only NS plz,non-smokers only please<br /><br />furnished r...,We live on a cul-de-sac in a small community o...,https://a0.muscache.com/pictures/4e4e34db-3155...,812128,...,5.00,5.00,5.00,NaN,f,6,1,5,0,0.09
4,258817,https://www.airbnb.com/rooms/258817,20250315060229,2025-03-15,city scrape,"ButterflyRoom-queen room, private bath",Morningstar House! Extended stay - 3+ months- ...,"I'm very near the Percy Priest Lake, natural ...",https://a0.muscache.com/pictures/eef19bc4-2743...,22296,...,4.91,4.74,4.73,NaN,f,8,0,7,1,0.60
5,289242,https://www.airbnb.com/rooms/289242,20250315060229,2025-03-15,city scrape,"MorningstarHouse, monthly room-",Morningstar House! Extended stay - price reduc...,The Morningstar House is in a quiet cul-de-sac...,https://a0.muscache.com/pictures/40a7ab72-20f7...,22296,...,4.78,4.47,4.69,NaN,f,8,0,7,1,0.47
6,319705,https://www.airbnb.com/rooms/319705,20250315060229,2025-03-15,city scrape,"SuperSweetSTUDIO, jacuzzi, private entrance",Huge fully furnished private room /studio apar...,"I'm very near the Percy Priest Lake, natural ...",https://a0.muscache.com/pictures/3470453/c7d82...,22296,...,4.69,4.63,4.58,NaN,f,8,0,7,1,0.31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9520,1368588735062432547,https://www.airbnb.com/rooms/1368588735062432547,20250315060229,2025-03-15,city scrape,Cozy Downtown Studio On Broadway,Welcome to your modern studio apt home on the ...,NaN,https://a0.muscache.com/pictures/miso/Hosting-...,492903135,...,NaN,NaN,NaN,NaN,f,4,4,0,0,NaN
9528,1370292228465842647,https://www.airbnb.com/rooms/1370292228465842647,20250315060229,2025-03-15,city scrape,I’m rarely there,Master bedroom on main floor of 2br/2ba townho...,NaN,https://a0.muscache.com/pictures/hosting/Hosti...,8962546,...,NaN,NaN,NaN,NaN,f,1,0,1,0,NaN
9554,1373976176884284666,https://www.airbnb.com/rooms/1373976176884284666,20250315060229,2025-03-15,city scrape,"Peaceful, Bright Queen Room | Central & Renovated",Keep it simple at this peaceful and centrally-...,NaN,https://a0.muscache.com/pictures/airflow/Hosti...,21646527,...,NaN,NaN,NaN,NaN,t,2,1,1,0,NaN
9567,1374694384021460568,https://www.airbnb.com/rooms/1374694384021460568,20250315060229,2025-03-15,city scrape,Mid-Mod Basement Retreat,NEWLY RENOVATED Modern/contemporary 2 BR/1 BA ...,NaN,https://a0.muscache.com/pictures/miso/Hosting-...,12774554,...,NaN,NaN,NaN,NaN,f,1,1,0,0,NaN


In [26]:
nash_review = pd.read_csv('../data/reviews.csv')

In [27]:
nash_list['price']=pd.Series(nash_list['price']).to_string()
type(nash_list['price'])

pandas.core.series.Series

In [28]:
len(set(nash_review))

6

In [29]:
nash_neighbors = pd.read_csv('../data/neighbourhoods.csv')
nash_neighbors

,neighbourhood_group,neighbourhood
0,NaN,District 1
1,NaN,District 10
2,NaN,District 11
3,NaN,District 12
4,NaN,District 13
5,NaN,District 14
6,NaN,District 15
7,NaN,District 16
8,NaN,District 17
9,NaN,District 18


In [30]:
##parksgpd['lng'] = parksgpd.geometry.centroid.x
##parksgpd['lat']= parksgpd.geometry.centroid.y
##parksgpd

In [78]:
zillow_list = pd.read_csv('../data/zillow.csv')

In [87]:
zillow_list

,RegionID,SizeRank,RegionName,RegionType,StateName,State,City,Metro,CountyName,2015-01-31,...,2024-07-31,2024-08-31,2024-09-30,2024-10-31,2024-11-30,2024-12-31,2025-01-31,2025-02-28,2025-03-31,2025-04-30
0,91982,1,77494,zip,TX,TX,Katy,"Houston-The Woodlands-Sugar Land, TX",Fort Bend County,1557.413520,...,2008.322823,2019.510812,2008.990192,1999.000695,1991.831855,1987.133409,1984.396568,1982.530426,1992.511067,1998.326857
1,91940,3,77449,zip,TX,TX,Katy,"Houston-The Woodlands-Sugar Land, TX",Harris County,1297.721295,...,1903.201526,1915.814721,1893.354338,1875.355315,1872.596561,1892.674418,1902.553639,1914.897081,1927.398395,1928.312343
2,91733,5,77084,zip,TX,TX,Houston,"Houston-The Woodlands-Sugar Land, TX",Harris County,1213.080633,...,1798.527327,1797.622832,1800.911804,1807.872389,1803.258361,1791.416816,1768.558354,1783.547286,1781.959060,1785.311992
3,93144,6,79936,zip,TX,TX,El Paso,"El Paso, TX",El Paso County,NaN,...,1516.142641,1508.063301,1500.805038,1485.205436,1491.904534,1505.402383,1534.994034,1554.616415,1549.810299,1518.400560
4,62093,7,11385,zip,NY,NY,New York,"New York-Newark-Jersey City, NY-NJ-PA",Queens County,NaN,...,2993.317633,2988.242148,2953.951035,2908.311413,2860.673857,2838.826917,2883.699434,2921.008142,2974.454518,2965.071455
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7317,86344,30470,64102,zip,MO,MO,Kansas City,"Kansas City, MO-KS",Jackson County,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1589.751960,1586.000000
7318,58956,30490,2876,zip,RI,RI,North Smithfield,"Providence-Warwick, RI-MA",Providence County,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,2255.910013,2236.505170,2268.402375,2297.222222
7319,61618,30490,10004,zip,NY,NY,New York,"New York-Newark-Jersey City, NY-NJ-PA",New York County,NaN,...,4563.087965,4534.430074,4483.314883,4403.330101,4371.507238,4329.240618,4359.792613,4475.881316,4530.669700,4618.611111
7320,91179,30490,76005,zip,TX,TX,Arlington,"Dallas-Fort Worth-Arlington, TX",Tarrant County,NaN,...,2030.587629,2007.140006,1978.581956,1921.595885,1940.733714,1915.469691,1919.755958,1900.543381,1907.209568,1921.465368


In [111]:
zillow_list.loc[zillow_list['City']== 'Nashville']

,RegionID,SizeRank,RegionName,RegionType,StateName,State,City,Metro,CountyName,2015-01-31,...,2024-07-31,2024-08-31,2024-09-30,2024-10-31,2024-11-30,2024-12-31,2025-01-31,2025-02-28,2025-03-31,2025-04-30
12,74101,15,37013,zip,TN,TN,Nashville,"Nashville-Davidson--Murfreesboro--Franklin, TN",Davidson County,1119.967929,...,1934.575940,1932.546290,1928.691310,1911.247988,1878.485364,1861.859260,1864.345966,1875.617022,1884.520976,1887.843159
166,74242,176,37211,zip,TN,TN,Nashville,"Nashville-Davidson--Murfreesboro--Franklin, TN",Davidson County,974.183647,...,1558.986481,1548.374318,1526.986090,1503.205083,1496.812194,1503.824173,1509.036065,1515.705622,1512.477114,1530.493139
1657,74178,1740,37115,zip,TN,TN,Nashville,"Nashville-Davidson--Murfreesboro--Franklin, TN",Davidson County,NaN,...,1504.517498,1516.465714,1506.479409,1515.435507,1505.485960,1514.737987,1514.708241,1532.739374,1536.187325,1522.991142
1658,74252,1741,37221,zip,TN,TN,Nashville,"Nashville-Davidson--Murfreesboro--Franklin, TN",Davidson County,NaN,...,1716.398503,1720.305525,1733.220823,1741.535599,1749.034831,1710.217674,1699.698147,1689.237242,1709.447656,1716.247941
1848,74238,1943,37207,zip,TN,TN,Nashville,"Nashville-Davidson--Murfreesboro--Franklin, TN",Davidson County,NaN,...,2204.249611,2185.810457,2181.963035,2161.386301,2150.777045,2174.060029,2180.251483,2188.925953,2186.900959,2220.505556
2057,74157,2166,37076,zip,TN,TN,Nashville,"Nashville-Davidson--Murfreesboro--Franklin, TN",Davidson County,NaN,...,1632.530846,1640.629799,1637.274732,1605.828035,1597.222447,1597.237315,1615.603384,1625.691698,1639.552714,1650.986555
2455,74240,2594,37209,zip,TN,TN,Nashville,"Nashville-Davidson--Murfreesboro--Franklin, TN",Davidson County,NaN,...,1875.479063,1888.072606,1873.094818,1874.974533,1857.869017,1869.548847,1845.920267,1840.603366,1826.655239,1832.551307
2981,74248,3187,37217,zip,TN,TN,Nashville,"Nashville-Davidson--Murfreesboro--Franklin, TN",Davidson County,NaN,...,1569.125785,1562.404350,1552.364713,1534.223049,1542.992354,1524.948233,1514.992987,1502.046505,1516.166359,1533.345303
3135,74245,3360,37214,zip,TN,TN,Nashville,"Nashville-Davidson--Murfreesboro--Franklin, TN",Davidson County,NaN,...,1663.871248,1677.739514,1671.397646,1658.844502,1628.894733,1618.672989,1623.993137,1637.291221,1666.188025,1671.495252
3923,74195,4310,37138,zip,TN,TN,Nashville,"Nashville-Davidson--Murfreesboro--Franklin, TN",Davidson County,NaN,...,2160.961860,2138.543989,2094.676884,2068.799277,2092.204702,2148.245754,2155.067420,2173.084371,2160.086058,2149.704762


In [113]:
zillow_list.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7322 entries, 0 to 7321
Data columns (total 133 columns):
 #    Column      Non-Null Count  Dtype  
---   ------      --------------  -----  
 0    RegionID    7322 non-null   int64  
 1    SizeRank    7322 non-null   int64  
 2    RegionName  7322 non-null   int64  
 3    RegionType  7322 non-null   object 
 4    StateName   7322 non-null   object 
 5    State       7322 non-null   object 
 6    City        7270 non-null   object 
 7    Metro       7301 non-null   object 
 8    CountyName  7322 non-null   object 
 9    2015-01-31  809 non-null    float64
 10   2015-02-28  838 non-null    float64
 11   2015-03-31  893 non-null    float64
 12   2015-04-30  909 non-null    float64
 13   2015-05-31  924 non-null    float64
 14   2015-06-30  926 non-null    float64
 15   2015-07-31  933 non-null    float64
 16   2015-08-31  936 non-null    float64
 17   2015-09-30  939 non-null    float64
 18   2015-10-31  945 non-null    float64
 19   2015

In [114]:
zori_list = pd.read_csv('../data/zori.csv')

In [123]:
zori_list.loc[zori_list['StateName']== 'TN']

,RegionID,SizeRank,RegionName,RegionType,StateName,2018-03-31,2018-04-30,2018-05-31,2018-06-30,2018-07-31,...,2024-07-31,2024-08-31,2024-09-30,2024-10-31,2024-11-30,2024-12-31,2025-01-31,2025-02-28,2025-03-31,2025-04-30
36,394902,37,"Nashville, TN",msa,TN,6837.0,7421.0,8141.0,8739.0,9184.0,...,8329.0,8653.0,8725.0,8721.0,8484.0,7972.0,7503.0,7240.0,7636.0,8242.0
43,394849,44,"Memphis, TN",msa,TN,4814.0,4975.0,5202.0,5420.0,5647.0,...,5526.0,5637.0,5536.0,5419.0,5211.0,4977.0,4768.0,4637.0,4763.0,4921.0
64,394753,65,"Knoxville, TN",msa,TN,4732.0,4902.0,5111.0,5232.0,5353.0,...,3388.0,3565.0,3651.0,3688.0,3652.0,3528.0,3412.0,3331.0,3501.0,3775.0
101,394460,102,"Chattanooga, TN",msa,TN,2346.0,2433.0,2556.0,2700.0,2860.0,...,2393.0,2533.0,2623.0,2649.0,2514.0,2311.0,2160.0,2101.0,2186.0,2268.0
162,394471,163,"Clarksville, TN",msa,TN,1435.0,1501.0,1581.0,1649.0,1700.0,...,1372.0,1409.0,1425.0,1419.0,1389.0,1329.0,1304.0,1298.0,1375.0,1475.0
165,394747,167,"Kingsport, TN",msa,TN,1698.0,1746.0,1816.0,1872.0,1926.0,...,992.0,1028.0,1072.0,1098.0,1112.0,1069.0,1024.0,968.0,1009.0,1073.0
221,394726,224,"Johnson City, TN",msa,TN,1010.0,1035.0,1085.0,1122.0,1154.0,...,607.0,646.0,657.0,668.0,655.0,619.0,570.0,548.0,575.0,621.0
242,394712,246,"Jackson, TN",msa,TN,578.0,586.0,613.0,626.0,635.0,...,610.0,638.0,659.0,670.0,663.0,638.0,616.0,604.0,607.0,625.0
303,394880,307,"Morristown, TN",msa,TN,630.0,642.0,669.0,689.0,701.0,...,539.0,576.0,607.0,625.0,622.0,591.0,559.0,528.0,530.0,551.0
329,394474,334,"Cleveland, TN",msa,TN,573.0,580.0,605.0,624.0,654.0,...,562.0,582.0,570.0,559.0,526.0,486.0,458.0,441.0,460.0,480.0
